In [2]:
# noexport

!typech efficacy_utils_betweendifficulties.ipynb

[NbConvertApp] Converting notebook efficacy_utils_betweendifficulties.ipynb to python
retention_utils.py:398: error: Name 'get_ipython' is not defined
efficacy_utils_betweendifficulties.py:40: error: Name 'get_condition_to_installs_for_random_assignment_abtest' already defined on line 27
efficacy_utils_betweendifficulties.py:159: error: Name 'get_ipython' is not defined
efficacy_utils_betweendifficulties.py:168: error: Name 'get_ipython' is not defined
efficacy_utils_betweendifficulties.py:185: error: Name 'get_ipython' is not defined
efficacy_utils_betweendifficulties.py:189: error: Name 'get_ipython' is not defined
efficacy_utils_betweendifficulties.py:203: error: Name 'condition_to_mlengths' is not defined


In [3]:
from retention_utils import *

<IPython.core.display.Javascript object>

In [4]:
import jsonmemo as jsonmemo_module
jsonmemo_module.set_lowmem(True)

In [4]:
#installs_with_experiment_vars = get_installs_with_experiment_vars()

In [5]:
#print(installs_with_experiment_vars[0])

In [5]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen'
  groups = ['survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [34]:


# def get_condition_to_installs_for_random_assignment_abtest():
#   abtest_name = 'difficulty_selection_screen_and_choose_difficulty_frequency'
#   groups = ['survey', 'nodefault_forcedchoice_userchoice', 'survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
#   condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
#   for k in list(condition_to_installs.keys()):
#     if k not in groups:
#       del condition_to_installs[k]
#   return condition_to_installs


In [35]:

#for install in installs_with_experiment_vars:
@msgpackmemo1arg
def get_domain_to_intervention_to_session_lengths_for_install(install):
  seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
  
  domain_to_session_to_intervention = {}
  for x in interventions_active_for_domain_and_session:
    if x.get('val') is None:
      continue
    interventions_active = json.loads(x['val'])
    if len(interventions_active) == 0:
      continue
    intervention_name = interventions_active[0]
    domain = x['key']
    session_id = x['key2']
    if domain not in domain_to_session_to_intervention:
      domain_to_session_to_intervention[domain] = {}
    domain_to_session_to_intervention[domain][session_id] = intervention_name
  
  domain_to_intervention_to_session_lengths = {}
  for x in seconds_on_domain_per_session:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_session')
      print(x)
      continue
    domain = x['key']
    session_id = x['key2']
    time_spent = x['val']
    if domain not in domain_to_session_to_intervention:
      continue
    intervention_name = domain_to_session_to_intervention[domain].get(session_id)
    if intervention_name is None:
      continue
    if domain not in domain_to_intervention_to_session_lengths:
      domain_to_intervention_to_session_lengths[domain] = {}
    if intervention_name not in domain_to_intervention_to_session_lengths[domain]:
      domain_to_intervention_to_session_lengths[domain][intervention_name] = []
    domain_to_intervention_to_session_lengths[domain][intervention_name].append(time_spent)
  return domain_to_intervention_to_session_lengths
  

In [36]:
# noexport

for x in get_collection_names():
  if 'per_day' in x and 'visits' not in x:
    print(x)
    break

edceda6041ef668213a96ba5_synced:seconds_on_domain_per_day


In [37]:
@msgpackmemo1arg
def get_epoch_to_domain_to_time_spent(install):
  seconds_on_domain_per_day = get_collection_for_install(install, 'synced:seconds_on_domain_per_day')
  output = {}
  for x in seconds_on_domain_per_day:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_day')
      print(x)
      continue
    domain = x['key']
    epoch = x['key2']
    seconds = x['val']
    if epoch not in output:
      output[epoch] = {}
    output[epoch][domain] = seconds
  return output

In [38]:
# noexport

condition_to_installs = get_condition_to_installs_for_random_assignment_abtest()


In [39]:
# noexport

for condition,installs in condition_to_installs.items():
  for install in installs:
    print(sorted(get_epoch_to_domain_to_time_spent(install).keys()))
    break

[1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1325, 1326, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336]
[1323, 1327, 1335]
[]
[1326]
[1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335]
[]


In [63]:
import math

def make_domain_to_daily_time_dataframe():
  output = []
  for condition,installs in condition_to_installs.items():
    #condition_to_lengths[condition] = []
    for install in installs:
      for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
#         total_time_spent = sum(domain_to_time_spent.values())
#         output.append({
#           'user': install,
#           'epoch': epoch,
#           'time': total_time_spent,
#           'logtime': math.log(total_time_spent),
#           'condition': condition,
#         })
        for domain,total_time_spent in  domain_to_time_spent.items():
          output.append({
            'domain': domain,
            'user': install,
            'epoch': epoch,
            'time': total_time_spent,
            'logtime': math.log(total_time_spent),
            'condition': condition,
          })
  return to_dataframe(output)

In [64]:
df = make_domain_to_daily_time_dataframe()

In [15]:
#print(df)

In [67]:
%Rpush df

In [50]:
#%%R
#install.library

In [65]:
%%R
library(lme4)
#library(sjPlot)
library(lmerTest)
library(stargazer)


In [18]:
# %%R

# df$user <- as.factor(df$user)
# #df$domain <- as.factor(df$domain)
# df$condition <- as.factor(df$condition)
# df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard"))
# df$epoch <- as.factor(df$epoch)
# df$logtime <- as.numeric(df$logtime)
# df$time <- as.numeric(df$time)
# summary(df)



In [68]:
%%R

df$user <- as.factor(df$user)
df$domain <- as.factor(df$domain)
df$condition <- as.factor(df$condition)
df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard", "survey", "nodefault_forcedchoice_userchoice"))
df$epoch <- as.factor(df$epoch)
df$logtime <- as.numeric(df$logtime)
df$time <- as.numeric(df$time)
summary(df)




              domain                             user            epoch       
 www.google.com  :  7272   9ec9126bfe989d527d87f56a:  4150   1319   :  6600  
 www.youtube.com :  5029   4923f5d68ea2b45c0e582fa9:  3924   1333   :  6599  
 mail.google.com :  4588   3d8f23613f3849ed266ace58:  3148   1328   :  6555  
 www.facebook.com:  3524   f9329b2116e3bd031cca0519:  2867   1329   :  6409  
 docs.google.com :  2407   ac3fb867e496b79bd34dff4a:  2807   1320   :  6382  
 drive.google.com:  1758   a99d3dda7e8757513429395b:  2708   1327   :  6292  
 (Other)         :209309   (Other)                 :214283   (Other):195050  
      time            logtime                                   condition    
 Min.   :    1.0   Min.   : 0.000   survey_nochoice_nothing          :36536  
 1st Qu.:   14.0   1st Qu.: 2.639   survey_nochoice_easy             :44561  
 Median :   54.0   Median : 3.989   survey_nochoice_medium           :31895  
 Mean   :  347.8   Mean   : 3.987   survey_nochoice_hard        

In [70]:
%%R

#results <- lmer(logtime ~ condition + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user), data = df)
results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
show(results)
show(summary(results))
class(results) <- "lmerMod"
stargazer(results)

Linear mixed model fit by REML ['lmerModLmerTest']
Formula: logtime ~ condition + (1 | user) + (1 | domain)
   Data: df
REML criterion at convergence: 895180.7
Random effects:
 Groups   Name        Std.Dev.
 domain   (Intercept) 0.8027  
 user     (Intercept) 0.3569  
 Residual             1.5388  
Number of obs: 233887, groups:  domain, 69212; user, 693
Fixed Effects:
                               (Intercept)  
                                 3.5137305  
             conditionsurvey_nochoice_easy  
                                 0.0098042  
           conditionsurvey_nochoice_medium  
                                -0.0567833  
             conditionsurvey_nochoice_hard  
                                 0.0006367  
                           conditionsurvey  
                                -0.0244560  
conditionnodefault_forcedchoice_userchoice  
                                -0.0675178  
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Fo

In [20]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      condition_to_lengths[condition].append(total_time_spent)
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [21]:
# noexport
# bad


condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for domain,intervention_to_session_lengths in get_domain_to_intervention_to_session_lengths_for_install(install).items():
      if domain != 'www.youtube.com':
        continue
      for intervention,session_lengths in intervention_to_session_lengths.items():
        for session_length in session_lengths:
          condition_to_lengths[condition].append(session_length)


In [22]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    all_lengths_for_user = []
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      all_lengths_for_user.append(total_time_spent)
    if len(all_lengths_for_user) == 0:
      continue
    condition_to_lengths[condition].append(np.median(all_lengths_for_user))
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [23]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      if 'www.facebook.com' not in domain_to_time_spent:
        continue
      time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      condition_to_lengths[condition].append(time_on_facebook)

In [29]:
# noexport
from scipy.stats import ttest_ind

condition_to_mlengths = {}
for condition,session_lengths in condition_to_lengths.items():
  print(condition, np.mean(session_lengths))
  condition_to_mlengths[condition] = np.mean(session_lengths) / 60

#print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_nothing']))
print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_easy']))
#print(ttest_ind(condition_to_lengths['survey_nochoice_easy'], condition_to_lengths['survey_nochoice_medium']))


survey 857.5116666666667
nodefault_forcedchoice_userchoice 886.2849264705883
survey_nochoice_hard 1181.743068391867
survey_nochoice_medium 1158.2426035502958
survey_nochoice_nothing 628.2053571428571
survey_nochoice_easy 1832.9417098445597
Ttest_indResult(statistic=-3.8441432304743293, pvalue=0.00012677234604202193)


In [25]:
from plot_utils import *


In [26]:
#from plot_utils import *
plot_dict_as_bar(condition_to_mlengths, title='Total minutes spent daily on sites', xlabel='Difficulty', ylabel='Minutes spent daily on sites', remap_labels={'survey_nochoice_nothing': 'Nothing', 'survey_nochoice_easy': 'Easy', 'survey_nochoice_medium': 'Medium', 'survey_nochoice_hard': 'Hard', 'survey': 'Experience Sampling', 'nodefault_forcedchoice_userchoice': 'Onboarding Choice'})

In [27]:
# for install in installs_with_experiment_vars:
#   #seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
#   #interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
#   res = get_domain_to_intervention_to_session_lengths_for_install(install)
#   #if len(res) > 0:
#   #  print(res)
#   #  break